<a href="https://colab.research.google.com/github/PyTorchLightning/lightning-flash/blob/master/flash_notebooks/text_classification.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this notebook, we'll go over the basics of lightning Flash by finetunig a TextClassifier on [IMDB Dataset](https://www.imdb.com/interfaces/).

# Finetuning

Finetuning consists of four steps:
 
 - 1. Train a source neural network model on a source dataset. For text classication, it is traditionally  a transformer model such as BERT [Bidirectional Encoder Representations from Transformers](https://arxiv.org/abs/1810.04805) trained on wikipedia.
As those model are costly to train, [Transformers](https://github.com/huggingface/transformers) or [FairSeq](https://github.com/pytorch/fairseq) libraries provides popular pre-trained model architectures for NLP. In this notebook, we will be using [tiny-bert](https://huggingface.co/prajjwal1/bert-tiny).

 
 - 2. Create a new neural network the target model. Its architecture replicates all model designs and their parameters on the source model, expect the latest layer which is removed. This model without its latest layers is traditionally called a backbone
 

- 3. Add new layers after the backbone where the latest output size is the number of target dataset categories. Those new layers, traditionally called head, will be randomly initialized while backbone will conserve its pre-trained weights from ImageNet.
 

- 4. Train the target model on a target dataset, such as Hymenoptera Dataset with ants and bees. However, freezing some layers at training start such as the backbone tends to be more stable. In Flash, it can easily be done with `trainer.finetune(..., strategy="freeze")`. It is also common to `freeze/unfreeze` the backbone. In `Flash`, it can be done with `trainer.finetune(..., strategy="freeze_unfreeze")`. If a one wants more control on the unfreeze flow, Flash supports `trainer.finetune(..., strategy=MyFinetuningStrategy())` where `MyFinetuningStrategy` is subclassing `pytorch_lightning.callbacks.BaseFinetuning`.

---
  - Give us a ⭐ [on Github](https://www.github.com/PytorchLightning/pytorch-lightning/)
  - Check out [Flash documentation](https://lightning-flash.readthedocs.io/en/latest/)
  - Check out [Lightning documentation](https://pytorch-lightning.readthedocs.io/en/latest/)
  - Join us [on Slack](https://join.slack.com/t/pytorch-lightning/shared_invite/zt-pw5v393p-qRaDgEk24~EjiZNBpSQFgQ)

### Setup  
Lightning Flash is easy to install. Simply ```pip install lightning-flash```

In [ ]:
%%capture
! pip install 'git+https://github.com/PyTorchLightning/lightning-flash.git#egg=lightning-flash[text]'

In [1]:
import flash
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier
from flash.text.classification.data import TextDeserializer
from flash.text.classification.tokenizers import TEXT_CLASSIFIER_TOKENIZERS

In [12]:
import datasets

In [19]:
df = datasets.load_dataset("csv", "data/imdb/train.csv", data_files={"train": "data/imdb/train.csv"})

Using custom data configuration data/imdb/train.csv-a4d22b10f26a6d19
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/data/imdb/train.csv-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [31]:
datasets.cached_path("data/imdb/train.csv")

'data/imdb/train.csv'

In [43]:
datasets.Dataset.from_csv("data/imdb/train.csv", keep_in_memory=True)

Using custom data configuration default-a4d22b10f26a6d19


FileNotFoundError: [Errno 2] No such file or directory: '/Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/dataset_info.json'

In [7]:
datamodule = TextClassificationData.from_csv(
    train_file="data/imdb/train.csv",
    input_fields="review",
    target_fields="sentiment",
    backbone="prajjwal1/bert-tiny",
    backbone_kwargs={"pretrained": False, "batch_size": 500},
)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/tokenizers (https://github.com/huggingface/tokenizers).
Using custom data configuration default-a4d22b10f26a6d19
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-3603c3c1f2d11b36.arrow





Tokenizer fit with `vocab_size=30522`, `max_length=512`, `batch_size=500`


In [8]:
datamodule.preprocess.tokenizer("hello")

{'input_ids': [2, 8687, 3], 'attention_mask': [1, 1, 1]}

In [9]:
tok = TEXT_CLASSIFIER_TOKENIZERS.get("prajjwal1/bert-tiny")(pretrained=True)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/tokenizers (https://github.com/huggingface/tokenizers).


In [11]:
tok[0]("hello")

{'input_ids': [101, 7592, 102], 'attention_mask': [1, 1, 1]}

In [3]:
model = TextClassifier(
    num_classes=2,
    backbone="prajjwal1/bert-tiny",
)

tok = TEXT_CLASSIFIER_TOKENIZERS.get("prajjwal1/bert-tiny")(pretrained=True)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Using 'prajjwal1/bert-tiny' provided by Hugging Face/tokenizers (https://github.com/huggingface/tokenizers).


TypeError: '>' not supported between instances of 'NoneType' and 'int'

###  1. Download the data
The data are downloaded from a URL, and save in a 'data' directory.

In [ ]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", 'data/')

<h2>2. Load the data</h2>

Flash Tasks have built-in DataModules that you can use to organize your data. Pass in a train, validation and test folders and Flash will take care of the rest.
Creates a TextClassificationData object from csv file.

In [4]:
datamodule = TextClassificationData.from_csv(
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    input_fields="review",
    target_fields="sentiment",
    backbone="prajjwal1/bert-tiny",
    backbone_kwargs={"pretrained": True},
)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Using custom data configuration default-a4d22b10f26a6d19
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/49796/.cache/huggingface/datasets/csv/default-a4d22b10f26a6d19/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-3603c3c1f2d11b36.arrow
Using custom data configuration default-44d9b23f3b65e5df
Reusing dataset csv (/Users/49796/.cache/huggingface/datasets/csv/default-44d9b23f3b65e5df/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/49796/.cache/huggingface/datasets/csv/default-44d9b23f3b65e5df/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-bcb6abcf8a2efd71.arrow
Using custom data configuration def

###  3. Build the model

Create the TextClassifier task. By default, the TextClassifier task uses a [tiny-bert](https://huggingface.co/prajjwal1/bert-tiny) backbone to train or finetune your model demo. You could use any models from [transformers - Text Classification](https://huggingface.co/models?filter=text-classification,pytorch)

Backbone can easily be changed with such as `TextClassifier(backbone='bert-tiny-mnli')`

In [12]:
model = TextClassifier(
    num_classes=datamodule.num_classes,
    backbone="prajjwal1/bert-tiny",
)
model.predict(["ciao"])

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).


/Users/49796/Documents/GitHub/lightning-flash/flash/core/model.py:594: LightningDeprecationWarning: The `LightningModule.datamodule` property is deprecated in v1.3 and will be removed in v1.5. Access the datamodule through using `self.trainer.datamodule` instead.
  elif getattr(self, "datamodule", None) is not None:


TypeError: string indices must be integers

###  4. Create the trainer. Run once on data

In [6]:
trainer = flash.Trainer(max_epochs=1, limit_train_batches=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


###  5. Fine-tune the model

The backbone won't be freezed and the entire model will be finetuned on the imdb dataset 

In [7]:
trainer.finetune(model, datamodule=datamodule, strategy="no_freeze")


  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | adapter       | DefaultAdapter | 4.4 M 
-------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


/Users/49796/miniconda3/envs/flash-dev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/49796/miniconda3/envs/flash-dev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/49796/miniconda3/envs/flash-dev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:322: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(l

Epoch 0: 100%|██████████| 626/626 [00:33<00:00, 18.54it/s, loss=0.796, v_num=3, train_accuracy_step=0.250, train_cross_entropy_step=0.796, val_accuracy=0.488, val_cross_entropy=1.250]


###  6. Test model

In [8]:
trainer.test(model, datamodule=datamodule)

/Users/49796/miniconda3/envs/flash-dev/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  rank_zero_deprecation(
/Users/49796/miniconda3/envs/flash-dev/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 100%|█████████▉| 623/625 [00:32<00:00, 21.38it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.504800021648407, 'test_cross_entropy': 1.2135802507400513}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 625/625 [00:32<00:00, 18.96it/s]


[{'test_accuracy': 0.504800021648407,
  'test_cross_entropy': 1.2135802507400513}]

###  7. Save it!

In [9]:
trainer.save_checkpoint("text_classification_model.pt")

# Predicting

### 1. Load the model from a checkpoint

In [10]:
model = TextClassifier.load_from_checkpoint("https://flash-weights.s3.amazonaws.com/text_classification_model.pt")

AttributeError: 'TextCSVDataSource' object has no attribute 'backbone_kwargs'

### 2a. Classify a few sentences! How was the movie?

In [11]:
predictions = model.predict([
    "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
    "The worst movie in the history of cinema.",
    "I come from Bulgaria where it 's almost impossible to have a tornado."
    "Very, very afraid"
    "This guy has done a great job with this movie!",
])
print(predictions)

TypeError: string indices must be integers

### 2b. Or generate predictions from a sheet file!

In [ ]:
datamodule = TextClassificationData.from_csv(
    predict_file="data/imdb/predict.csv",
    input_fields="review",
)
predictions = flash.Trainer().predict(model, datamodule=datamodule)
print(predictions)

<code style="color:#792ee5;">
    <h1> <strong> Congratulations - Time to Join the Community! </strong>  </h1>
</code>

Congratulations on completing this notebook tutorial! If you enjoyed it and would like to join the Lightning movement, you can do so in the following ways!

### Help us build Flash by adding support for new data-types and new tasks.
Flash aims at becoming the first task hub, so anyone can get started to great amazing application using deep learning. 
If you are interested, please open a PR with your contributions !!! 


### Star [Lightning](https://github.com/PyTorchLightning/pytorch-lightning) on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

* Please, star [Lightning](https://github.com/PyTorchLightning/pytorch-lightning)

### Join our [Slack](https://join.slack.com/t/pytorch-lightning/shared_invite/zt-pw5v393p-qRaDgEk24~EjiZNBpSQFgQ)!
The best way to keep up to date on the latest advancements is to join our community! Make sure to introduce yourself and share your interests in `#general` channel

### Interested by SOTA AI models ! Check out [Bolt](https://github.com/PyTorchLightning/lightning-bolts)
Bolts has a collection of state-of-the-art models, all implemented in [Lightning](https://github.com/PyTorchLightning/pytorch-lightning) and can be easily integrated within your own projects.

* Please, star [Bolt](https://github.com/PyTorchLightning/lightning-bolts)

### Contributions !
The best way to contribute to our community is to become a code contributor! At any time you can go to [Lightning](https://github.com/PyTorchLightning/pytorch-lightning) or [Bolt](https://github.com/PyTorchLightning/lightning-bolts) GitHub Issues page and filter for "good first issue". 

* [Lightning good first issue](https://github.com/PyTorchLightning/pytorch-lightning/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)
* [Bolt good first issue](https://github.com/PyTorchLightning/lightning-bolts/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)
* You can also contribute your own notebooks with useful examples !

### Great thanks from the entire Pytorch Lightning Team for your interest !

<img src="https://raw.githubusercontent.com/PyTorchLightning/lightning-flash/18c591747e40a0ad862d4f82943d209b8cc25358/docs/source/_static/images/logo.svg" width="800" height="200" />